In [2]:
# import os
# import sys

# cwd = os.getcwd()
# project_root = os.path.abspath(os.path.join(cwd, '..'))
# sys.path.append(project_root)

import pyoptflight as pof
from pyoptflight import initialize as optinit
from pyoptflight import plotting as optplot
import numpy as np

In [14]:
kerbin = pof.Body("Kerbin")

vehicle = pof.Stage.load_vehicle('mintoc_multi')

config = pof.SolverConfig(landing=False, 
                          T_min=0,
                          T_max = 700,
                          max_iter=250, 
                          solver_tol=1e-4, 
                          N=100, 
                          T_init=100,
                          q_max = 100,
                          integration_method='RK4')

# xf = pof.LatLngBoundary(lat=0,
#                         lng=0,
#                         alt=0,
#                         v_eps=1e-6,
#                         ub_lng=-180,
#                         lb_lng=180,
#                         body=kerbin,
#                         config=config)

xf = pof.LatLngBound(lat=0, lng=0, alt=0, vel = 1e-6, ERA0=np.pi, atti=None, atti_range=((-np.pi/2, np.pi/2), (0, 0)))

x0 = pof.KeplerianBoundary(i=np.deg2rad(60),
                           Ω=np.deg2rad(15),
                           ω=0,
                           ha=80,
                           hp=80,
                        #    ν = np.deg2rad(100),
                           body=kerbin)

# single_stage_solver = pof.Solver(kerbin, [single_stage], config, x0, xf)

multi_stage_solver = pof.Solver(kerbin, vehicle, config, x0, xf)

In [15]:
kerbin

Body(
    r_0      = 600,
    g_0      = 0.00981,
    mu       = 3531.6,
    omega_0  = 0.0002915709,
    atm      = Atmosphere(
        rho_0           = 1.225,
        H               = 5.6,
        gamma           = 1.4,
        Rg              = 287.05,
        C_T             = 288.15,
        cutoff_altitude = 75,
        color           = steelblue
    ),
    meshpath = mesh_images/Kerbin.jpg
)

In [16]:
xf

LatLngBound(
    lat        = 0,
    lng        = 0,
    alt        = 0,
    vel        = 1e-06,
    f          = None,
    atti       = None,
    atti_range = ((-1.5707963267948966, 1.5707963267948966), (0, 0)),
    ERA0       = 3.141592653589793,
    ERA0_range = (0, 6.283185307179586)
)

In [17]:
xb = xf.get_xb(multi_stage_solver)
keys = ['x', 'y', 'z', 'vx', 'vy', 'vz', 'f', 'psi', 'theta']
for i in range(0, 9):
    print(f"{keys[i]}: [{xb['lbx'][i]:.6f}, {xb['ubx'][i]:.6f}]")

3.141592653589793 3.141592653589793
x: [-600.000000, -600.000000]
y: [0.000000, 0.000000]
z: [0.000000, 0.000000]
vx: [-0.174943, 0.174943]
vy: [-0.174943, -0.000001]
vz: [0.000000, 0.000000]
f: [0.000000, 1.000000]
psi: [-3.141593, 3.141593]
theta: [0.000000, 0.000000]


In [ ]:
vehicle[0].prop.Isp_SL = 350
vehicle[0].prop.F_SL = 0.5
vehicle[0].aero.C_L = 0.01

vehicle[1].prop.Isp_SL = 350
vehicle[1].prop.F_SL = 0.5
vehicle[1].aero.C_L = 0.01

vehicle[2].prop.Isp_SL = 350
vehicle[2].prop.F_SL = 0.5
vehicle[2].aero.C_L = 0.01


In [ ]:
multi_stage_solver.create_nlp()

In [ ]:
multi_stage_solver.initialize_from_func(optinit.gravity_turn, {'skew': False})
multi_stage_solver.stats()
multi_stage_solver.fatrop_solve()

In [ ]:
multi_stage_solver.extra_opts['solver'] = 'ipopt'
multi_stage_solver.T_init = [multi_stage_solver.config.T_init]
multi_stage_solver.initialize_from_func(optinit.gravity_turn, {'skew': False})
multi_stage_solver.stats()
multi_stage_solver.fatrop_solve()

In [ ]:
multi_stage_solver.extra_opts['solver'] = 'ipopt'

In [ ]:
multi_stage_solver.nlpresult

In [ ]:
# RK4 base tol: 170, 74, 53, 384, 85, DNF (max iter) # No warm_start = False/no
# RK4 base tol: 170, 101ty, 139ty, 104 fn
# CVODES base tol: 189fn, 169tn, 119ty
# spral: 183, DNFty
# ma: Bugged!
# pardisomkl: MIA
# pardiso: Bugged! 

In [ ]:
import casadi as ca
J = ca.jacobian(ca.vertcat(*multi_stage_solver.G), multi_stage_solver.V)
sparsity_pattern = J.sparsity()

In [ ]:
print(sparsity_pattern)

In [ ]:
fig = optplot.plot_solutions(multi_stage_solver, 
                             show_orbit=True, 
                             size=(600, 600), 
                             ctrl_colorscale=True, 
                             ctrl_markers=True,
                             indices=[-1])
fig.show()

$$
\vec{D} = -\frac{1}{2}\rho C_D A ||\vec{v}||^2 \frac{\vec{v}}{||\vec{v}||}\\

\vec{D} = -\frac{1}{2}\rho C_D A ||\vec{v}|| \vec{v}\\

\vec{D} = -\frac{1}{2}\rho C_D A \sqrt{v_x^2+v_y^2+v_z^2} \vec{v}\\
$$


$$
\vec{L} = \frac{1}{2}\rho C_L A ||\vec{v}||^2 \frac{\vec{v}}{||\vec{v}||}\\
$$


$$
\hat{l} = \frac{\vec{v}\times(\vec{v}\times\hat{d})}{||\vec{v}\times(\vec{v}\times\hat{d}) + \epsilon||}
$$

$$
\text{proj}_{\vec{v}}\hat{d} = \frac{\hat{d}\cdot\vec{v}}{\vec{v}\cdot\vec{v}}\vec{v} = \frac{\hat{d}\cdot\vec{v}}{||\vec{v}||^2}\vec{v}
$$

$$
\hat{d} - \text{proj}_{\vec{v}}\hat{d} \\
\hat{d} - \frac{\hat{d}\cdot\vec{v}}{||\vec{v}||^2}\vec{v}
$$

$$
\vec{L} = \frac{1}{2}\rho C_L A ||\vec{v}||^2 \left(\hat{d} - \frac{\hat{d}\cdot\vec{v}}{||\vec{v}||^2}\vec{v}\right)
$$

$$
\hat{v}\times\frac{(\hat{v}\times\hat{d})}{\sin{\alpha}}
$$

In [ ]:
xp = x*np.cos(angle) + np.cross(axis, x)*np.sin(angle) + axis*np.dot(axis, x)*(1 - np.cos(angle))

In [ ]:
anlge = np.arctan2(np.dot(np.cross(a, b), axis), np.dot(a, b))

In [ ]:
np.arctan2(0, 0)

One idea is to use arccos to find the AoA then find the axis of rotation between d and v via cross product (can be zero vector) and then rotate d by pi/2 - AoA about axis using the above formula for vector rotation. 

In [ ]:
import casadi as ca
import numpy as np

vx = ca.SX.sym("vx")
vy = ca.SX.sym("vy")
vz = ca.SX.sym("vz")
v = ca.vertcat(vx, vy, vz)
dx = ca.SX.sym("dx")
dy = ca.SX.sym("dy")
dz = ca.SX.sym("dz")
d = ca.vertcat(dx, dy, dz)

n = ca.cross(v, ca.cross(d, v))
norm = ca.norm_2(n)

dtest = [0, -np.sin(0.0001), np.cos(0.0001)]
vtest = [0, 0, 1]

ftrue = ca.Function("ftrue",[vx, vy, vz, dx, dy, dz],[ca.vertcat(0, 0, 0)]) # True
ffalse = ca.Function("ffalse",[vx, vy, vz, dx, dy, dz],[n/norm]) # False
f_cond = ca.Function.if_else('f_cond', ftrue, ffalse)
func = ca.Function("func", [vx, vy, vz, dx, dy, dz], [f_cond(norm < 1e-15, vx, vy, vz, dx, dy, dz)])

In [ ]:
func(*vtest, *dtest)

In [ ]:
test_array = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
np.insert(test_array, [1, 2, 3], np.zeros((3)), axis=0)

In [ ]:
t1 = [1, 2, 3]
np.cumsum(t1)
sum(t1[0:0])

In [ ]:
np.append(test_array, np.array([np.zeros((3))]), axis=0)

In [ ]:
import numpy as np
test_arr = [[0, 1, 2, 3], [-1, 4, -2, 5]]
[1, *np.min(test_arr, axis=0).tolist()]
